In [1]:
%tensorflow_version 1.x
!pip install numpy==1.17.4
%load_ext tensorboard

TensorFlow 1.x selected.
     |████████████████████████████████| 20.0MB 1.6MB/s 
ERROR: tensorflow 1.15.2 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.4
    Uninstalling numpy-1.18.4:
      Successfully uninstalled numpy-1.18.4


In [2]:
from google.colab import drive
 
drive.mount('/gdrive')

%cd /gdrive/'My Drive'/TensorFlow

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/TensorFlow


In [3]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -qq Cython contextlib2 pillow lxml matplotlib pycocotools

Selecting previously unselected package python-bs4.
(Reading database ... 144429 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

In [0]:
# !git clone https://github.com/tensorflow/models.git


In [5]:
%ls

models/  ngrok-stable-linux-amd64.zip    ngrok-stable-linux-amd64.zip.2
ngrok    ngrok-stable-linux-amd64.zip.1  workspace/


In [0]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv

import re
import os
import io
import glob
import shutil
import urllib.request
import tarfile
import xml.etree.ElementTree as ET

import tensorflow.compat.v1 as tf
import cv2 

from PIL import Image
from collections import namedtuple, OrderedDict

from google.colab import files

In [7]:
print(tf.__version__)

1.15.2


In [8]:
%cd /gdrive/'My Drive'/TensorFlow/models/research/
# compils the proto buffers
!protoc object_detection/protos/*.proto --python_out=.
# exports PYTHONPATH environment var with research and slim paths
os.environ['PYTHONPATH'] += ':./:./slim/'

/gdrive/My Drive/TensorFlow/models/research


In [9]:
# testing the model builder
!python3 object_detection/builders/model_builder_test.py

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_wi

In [0]:
# Some models to train on
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
    },
    'faster_rcnn_inception': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
    },
    'mask_rcnn': {
        'model_name': 'mask_rcnn_inception_v2_coco_2018_01_28',
    },
}

# Select a model from `MODELS_CONFIG`.
# I chose ssd_mobilenet_v2 for this project, you could choose any
selected_model = 'faster_rcnn_inception'

In [11]:
%cd /gdrive/'My Drive'/TensorFlow/
%ls

/gdrive/My Drive/TensorFlow
models/  ngrok-stable-linux-amd64.zip    ngrok-stable-linux-amd64.zip.2
ngrok    ngrok-stable-linux-amd64.zip.1  workspace/


In [0]:

#the distination folder where the model will be saved
#change this if you have a different working dir
DEST_DIR = '/gdrive/My Drive/TensorFlow/models/research/pre_trained_model/' + selected_model

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

#selecting the model
MODEL_FILE = MODEL + '.tar.gz'

#creating the downlaod link for the model selected
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#checks if the model has already been downloaded, download it otherwise
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#unzipping the model and extracting its content
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# creating an output file to save the model while training
os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

In [0]:
%ls /gdrive/'My Drive'/TensorFlow/models/research/pre_trained_model/

faster_rcnn_inception/  sperm_tracker_rcnn/  ssd_mobilenet_v2/


In [0]:
%tensorboard --logdir /gdrive/'My Drive'/TensorFlow/models/research/training

In [15]:
%cd /gdrive/'My Drive'/TensorFlow/models/research

/gdrive/My Drive/TensorFlow/models/research


In [0]:
!python3 object_detection/model_main.py \
    --pipeline_config_path=/gdrive/My\ Drive/TensorFlow/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_coco.config \
    --model_dir=training/

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0510 08:40:52.209082 139814105151360 module_wrapper.py:139] From /gdrive/My Drive/TensorFlow/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0510 08:40:53.542028 139814105151360 module_wrapper.py:139] From /gdrive/My Drive/TensorFlow/models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0510 08:40:53.542250 139814105151360 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W0510 08:40:53.542412 139814105151360 module_wrap

In [0]:
#dir where the model will be saved
output_directory = './fine_tuned_model'

lst = os.listdir('training')
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join('training', last_model)

In [22]:
!python /gdrive/'My Drive'/TensorFlow/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/gdrive/My\ Drive/TensorFlow/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_coco.config \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0514 16:46:25.512727 140301182351232 module_wrapper.py:139] From /gdrive/My Drive/TensorFlow/models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0514 16:46:25.521242 140301182351232 module_wrapper.py:139] From /gdrive/My Drive/TensorFlow/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0514 16:46:25.525878 140301182351232 module_wrapper.py:139] From /gdrive/My Drive/TensorFlow/models/research/object_detection/exporter.py:121: The name tf.placeholder 